#### Understanding one-hot vectors
- Keras `to_categorical()` function to create one-hot vectors. The to_categorical() function expects a sequence of integers as the input. Therefore, a word2index dictionary is provided which can be used to convert a word to an integer. 

In [3]:
from keras.utils import to_categorical


In [7]:
# Create a list of words and convert them to indices
word2index = {"I":0, "like":1, "cats":2}
words = ["I", "like", "cats"]
word_ids = [word2index[w] for w in words]
print(word_ids)

[0, 1, 2]


In [10]:
# Create onehot vectors using to_categorical function
onehot_1 = to_categorical(word_ids)
print(onehot_1)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
# Print words and their corresponding onehot vectors
print([(w,ohe.tolist()) for w,ohe in zip(words, onehot_1)])

[('I', [1.0, 0.0, 0.0]), ('like', [0.0, 1.0, 0.0]), ('cats', [0.0, 0.0, 1.0])]


In [12]:
# Create onehot vectors with a fixed number of classes and print the result
onehot_2 = to_categorical(word_ids,num_classes=5)

print([(w,ohe.tolist()) for w,ohe in zip(words, onehot_2)])


[('I', [1.0, 0.0, 0.0, 0.0, 0.0]), ('like', [0.0, 1.0, 0.0, 0.0, 0.0]), ('cats', [0.0, 0.0, 1.0, 0.0, 0.0])]


#### In real-world problems, the vocabulary size can grow very large (e.g. more than hundred thousand) so its important to set `num_classes` 

In [15]:
def compute_onehot_length(words, word2index):
    """compute_onehot_length() that generates one-hot vectors for a given list of words
    and computes the length of those vectors. """
    # Create word IDs for words
    word_ids = [word2index[w] for w in words]
    # Convert word IDs to onehot vectors
    onehot = to_categorical(word_ids)
    # Return the length of a single one-hot vector
    return onehot.shape[1]

word2index = {"He":0, "drank": 1, "milk": 2}
# Compute and print onehot length of a list of words
print(compute_onehot_length(["He", "drank", "milk"], word2index))

3


In [17]:
word2index= {'He': 6,
             'I': 0,
             'We': 3,
             'cats': 2,
             'dogs': 5,
             'hates': 7,
             'like': 4,
             'rabbits': 8}

words_1 = ["I", "like", "cats", "We", "like", "dogs", "He", "hates", "rabbits"]
# Call compute_onehot_length on words_1
length_1 = compute_onehot_length(words_1, word2index)

words_2 = ["I", "like", "cats", "We", "like", "dogs", "We", "like", "cats"]
# Call compute_onehot_length on words_2
length_2 = compute_onehot_length(words_2, word2index)

# Print length_1 and length_2
print("length_1 =>", length_1, " and length_2 => ", length_2)

length_1 => 9  and length_2 =>  6


#### Deep learning models expect one-hot vectors to always have the same length. Leaving `num_classes` argument unchecked can lead to ill-defined inputs and various compilation errors when fed to Keras models.

### Text reversing model - Encoder
- Creating a simple text reversing model is a great method to understand the mechanics of encoder decoder models and how they connect.
- **Task** : define the words2onehot() helper function. The words2onehot() function should take in a list of words and a dictionary word2index and convert the list of words to an array of one-hot vectors. 


In [18]:
word2index = {'I': 0, 'cats': 2, 'like': 1}

import numpy as np

def words2onehot(word_list, word2index):
  # Convert words to word IDs
  word_ids = [word2index[w] for w in word_list]
  # Convert word IDs to onehot vectors and return the onehot array
  onehot = to_categorical(word_ids, num_classes=3)
  return onehot

words = ["I", "like", "cats"]
# Convert words to onehot vectors using words2onehot
onehot = words2onehot(words, word2index)
# Print the result as (<word>, <onehot>) tuples
print([(w,ohe.tolist()) for w,ohe in zip(words, onehot)])

[('I', [1.0, 0.0, 0.0]), ('like', [0.0, 1.0, 0.0]), ('cats', [0.0, 0.0, 1.0])]


#### a helper function that converts words to onehot vectors which will be fed to the encoder function.

- The encoder feeds on the one-hot vectors produced by the words2onehot() function.
- The encoder() function takes in a set of one-hot vectors and converts them to a list of word ids. 


In [20]:
# the encoder of a text reversing model.

word2index = {'We': 0, 'dogs': 2, 'like': 1}

def encoder(onehot):
  # Get word IDs from onehot vectors and return the IDs
  word_ids = np.argmax(onehot, axis=1)
  return word_ids

# Define "We like dogs" as words
words = ['We', 'like', 'dogs']
# Convert words to onehot vectors using words2onehot
onehot = words2onehot(words, word2index)
# Get the context vector by using the encoder function
context = encoder(onehot)
print(context)

[0 1 2]


####  implement the decoder part of the text reversing model, which will convert the context vector from the encoder to reversed words. 
- **Task** : defining two functions onehot2words() and decoder(). The onehot2words() function takes in a list of ids and a dictionary index2word and converts an array of one-hot vectors to a list of words. The decoder() function takes in the context vector (i.e., list of word ids) and converts it to the reversed list of words. 

In [21]:
index2word = {0: 'We', 1: 'like', 2: 'dogs'}

# Define the onehot2words function that returns words for a set of onehot vectors
def onehot2words(onehot, index2word):
  ids = np.argmax(onehot, axis=1)
  res = [index2word[id] for id in ids]
  return res
# Define the decoder function that returns reversed onehot vectors
def decoder(context_vector):
  word_ids_rev = context_vector[::-1]
  onehot_rev = to_categorical(word_ids_rev, num_classes=3)
  return onehot_rev
# Convert context to reversed onehot vectors using decoder
onehot_rev = decoder(context)
# Get the reversed words using the onehot2words function
reversed_words = onehot2words(onehot_rev, index2word)
print(reversed_words)

['dogs', 'like', 'We']
